# Standard libraries

Like pandapower, also pandapipes provides different standard libraries to simplify the workflow. In comparison with pandapower, pandapipes does not only provide a component library, but also a fluid library. After the pandapipes installation, this library already contains several fluids. Of course, it is also possible to define new fluids and add them to the library.

This tutorial demonstrates the usage with the libraries and shows, how components and fluids can be loaded and saved. We start with a tutorial on the component library. This library contains pipes and pumps.


## pandapipes component library

With the function "get_data", which is defined in the module std_type_toolbox, all entries in the library can be displayed. Necessary parameters include the path to the library file and the type of the component. In the following section, the pipe entries are plotted. 

In [1]:
from pandapipes.std_types.std_type_class import get_data
print(get_data(r'../pandapipes/std_types/library/Pipe.csv', 'pipe'))

hp.pandapower.plotting.plotly.traces - INFO: Failed to import plotly - interactive plotting will not be available


std_type                 80_GGG 100_GGG 125_GGG 150_GGG 200_GGG 250_GGG  \
nominal_width_mm             80     100     125     150     200     250   
outer_diameter_mm          98.0   118.0   144.0   170.0   222.0   274.0   
inner_diameter_mm          86.0   105.8   131.6   157.0   208.0   259.0   
standard_dimension_ratio  16.33   19.34   23.23   26.15   31.71   36.53   
material                    GGG     GGG     GGG     GGG     GGG     GGG   

std_type                 300_GGG 350_GGG 400_GGG 500_GGG 600_GGG  \
nominal_width_mm             300     350     400     500     600   
outer_diameter_mm          326.0   378.0   429.0   532.0   635.0   
inner_diameter_mm          310.0   361.0   411.0   512.0   613.0   
standard_dimension_ratio   40.75   44.47   47.67    53.2   57.73   
material                     GGG     GGG     GGG     GGG     GGG   

std_type                 20_PE_100_SDR_11 25_PE_100_SDR_11 32_PE_100_SDR_11  \
nominal_width_mm                       20               25   

The csv files can be extended. We will now create an example network using components from the library.

In [2]:
import pandapipes as pp
net = pp.create_empty_network()
junction1 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Connection to External Grid")
junction2 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction")
junction3 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction to sink")
medium_pressure_grid = pp.create_ext_grid(net, junction=junction1, fluid="water", p_bar=1, t_k=293.15, name="Grid Connection")
sink = pp.create_sink(net, junction=junction3, mdot_kg_per_s=23.05, name="Sink")

In the code above, a pipe connection is missing between junction1 and junction2. We load the pipe type "300_GGG" from the library using the function "create_pipe". Compared to the function "create_pipe_from_parameters" a parameter with a library reference can be specified.

In [3]:
pipe1 = pp.create_pipe(net, from_junction=junction2, to_junction=junction3, length_km=1, std_type="300_GGG", name="Pipe 1")

Between the external grid and the pipe, there is also a connection missing. This one will be defined using a pump component. The pressure difference is defined based on the underlying pump curve.

In [4]:
P1 = pp.create_pump(net, from_junction=junction1, to_junction=junction2, std_type="P1")

The network can be computed as usual.

In [5]:
pp.pipeflow(net)
net.res_pump

,deltap_bar,v_mean_m_per_s,p_from_bar,p_to_bar,t_from_k,t_to_k,mdot_from_kg_per_s,mdot_to_kg_per_s,vdot_norm_m3_per_s,compr_power_mw
0,3.995041,2.940182,1.0,4.995041,293.15,293.15,23.05,-23.05,0.023092,0.009225


The pump raises the pressure to 4 bar. We can check this by having a look at the pump curve.

In [6]:
print(get_data(r'../pandapipes/std_types/library/Pump/P1.csv', 'pump'))

   Vdot_m3ph  p_bar  degree
0        0.0    6.1     2.0
1       19.0    5.8     NaN
2       83.0    4.0     NaN


We notice that the pressure rais of 4 bar corresponds to a volume flow of 83 m^3/h. This volume flow is also defined at the sink.

## The fluid library

In the introductory example, the fluid is added via a parameter of the create_empty_network function. Internally, pandapipes fetches the corresponding fluid properties, which are saved for the specified fluid. Some of the predefined fluids are

- air
- hydrogen
- lgas
- hgas
- water
- methane

For the calculation of pressure and flow velocity the density and viscosity has to be known. In case of gas calculations, these fluid properties can be dependend on temperature. If the temperature is calculated, further material properties may be needed, for example the heat capacity of the fluid.

An overview of defined fluid properties is given in the directory pandapipes\properties. In the directory of each fluid, you can find the fluid properties. Each property is described by a text file. This file contains data points. During the calculation, pandapipes interpolates between these points.

If you want to know which properties are defined in the current net container, you can use the function get_property().

In [8]:
net.fluid['water'].get_property("density", 295)

array(997.746)

The second parameter is the temperature, at which the density is displayed.

## Defining new fluids

New fluids can be created or existing fluids modified. The mentioned fluids so far were dependent on temperature. We will now create an incompressible fluid with constant material properties and add it to the net.

In a first step, the material properties are created. These properties are then added to a fluid container. The data structures used to describe properties and fluids are pythin classes.

In [9]:
import pandapipes.properties.fluids as fl
density_const = fl.FluidPropertyConstant(900)
viscosity_const = fl.FluidPropertyConstant(0.002)
heat_capacity_const = fl.FluidPropertyConstant(4000)

The three properties density, viscosity and heat capacity have to be defined. For every property, an object of the class FluidPropertyConstant is created and the constant property value is used in the object constructor. All properties have to be entered using SI units.

Although constant properties are added here, pandapipes also provides classes for inter- and extrapolating properties.

Afterwards, a fluid container is created and the properties are added.

In [10]:
water_const =  fl.Fluid("water_const","liquid")
water_const.add_property(property_name="density",prop=density_const)
water_const.add_property(property_name="viscosity",prop=viscosity_const)
water_const.add_property(property_name="heat_capacity",prop= heat_capacity_const)

The fluid can now be used to calculate incompressible media. With the command ...:

In [11]:
fl._add_fluid_to_net(net,water_const)

... the fluid is added to the net container. By plotting again the fluid properties, we see that the old fluid has been replaced. 

In [12]:
net.fluid

{'water': Fluid water (liquid) with properties:
    - density (InterExtra)
    - viscosity (InterExtra)
    - heat_capacity (InterExtra)
    - molar_mass (Constant)
    - compressibility (Linear)
    - der_compressibility (Constant),
 'water_const': Fluid water_const (liquid) with properties:
    - density (Constant)
    - viscosity (Constant)
    - heat_capacity (Constant)}